In [1]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text
class AdventureWorksAnalyzer:
    """Classe pour analyser les données d'AdventureWorks via SQLAlchemy"""
    
    def __init__(self, db_name="Adventureworks", user="passion", password="secret", host="localhost", port="5432"):
        """Initialise la connexion à la base de données PostgreSQL"""
        self.db_name = db_name
        self.user = user
        self.password = password
        self.host = host
        self.port = port

        # Construction de l'URL de connexion
        self.db_url = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}"
        self.engine = create_engine(self.db_url)

        # Test de connexion
        self.test_connection()
    
    def test_connection(self):
        """Teste la connexion à la base de données via SQLAlchemy"""
        try:
            with self.engine.connect() as connection:
                version = connection.execute(text("SELECT version();")).scalar()
                print(f"Connexion réussie à PostgreSQL via SQLAlchemy: {version}")
        except SQLAlchemyError as e:
            print(f"Erreur de connexion via SQLAlchemy: {e}")
            raise

    def execute_query(self, query):
        """Exécute une requête SQL et retourne les résultats sous forme de DataFrame"""
        engine = create_engine(f"postgresql://{self.user}:{self.password}@{self.host}:{self.port}/{self.db_name}")
       # Exécuter la requête et charger dans un DataFrame
        df = pd.read_sql_query(query, engine)  
        return df


In [2]:
analyzer = AdventureWorksAnalyzer()

Connexion réussie à PostgreSQL via SQLAlchemy: PostgreSQL 14.17 (Homebrew) on aarch64-apple-darwin24.2.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit


In [7]:
query_sales_cost = """
SELECT 
    EXTRACT(YEAR FROM ModifiedDate) AS year,
    SUM(OrderQty * UnitPrice * (1 - UnitPriceDiscount)) AS total
FROM Sales.SalesOrderDetail
GROUP BY year;
"""
df = analyzer.execute_query(query_sales_cost)
df

,year,total
0,2011.0,1.264167e+07
1,2012.0,3.352430e+07
2,2013.0,4.362248e+07
3,2014.0,2.005793e+07


In [5]:
query = """
SELECT 
  ST.TerritoryID,
  CONCAT(ST.Name, ', ', ST.CountryRegionCode) AS Territories,
  (SUM(SOH.TotalDue) / SUM(SOD.OrderQty)) AS AverageOrderValue,
  SUM(SOH.TotalDue) AS Totalsales
FROM Sales.SalesTerritory ST
INNER JOIN Sales.SalesOrderHeader SOH ON ST.TerritoryID = SOH.TerritoryID
INNER JOIN Sales.SalesOrderDetail SOD ON SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY ST.TerritoryID, CONCAT(ST.Name, ', ', ST.CountryRegionCode)
ORDER BY Totalsales DESC;
"""

In [6]:
df = analyzer.execute_query(query)
df

,territoryid,territories,averageordervalue,totalsales
0,4,"Southwest, US",11790.823548,6.968966e+08
1,6,"Canada, CA",10671.502465,5.269695e+08
2,1,"Northwest, US",11181.402956,4.112073e+08
3,3,"Central, US",13497.109006,2.630991e+08
4,2,"Northeast, US",12788.958043,2.537713e+08
5,5,"Southeast, US",11996.156835,2.264275e+08
6,7,"France, FR",9954.701490,1.981583e+08
7,10,"United Kingdom, GB",9314.315222,1.872084e+08
8,8,"Germany, DE",7050.005756,9.265823e+07
9,9,"Australia, AU",3857.974316,7.057392e+07
